# Scratch

Scratch notebook for testing code.

In [1]:
import sys
sys.path.append('/netapp/home/mincheol/scVI')
sys.path.append('/netapp/home/mincheol/scVI-extensions')

In [2]:
import torch
import imp

In [3]:
import os

import numpy as np
from sklearn.manifold import TSNE
import matplotlib
matplotlib.use('AGG')
import matplotlib.pyplot as plt
import seaborn as sns

from scvi.metrics.clustering import entropy_batch_mixing, get_latent
from scvi.models import VAE, SVAEC
from scvi.inference import VariationalInference

/netapp/home/mincheol/anaconda3/envs/scvi/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
import scvi_extensions.dataset as ds

In [5]:
imp.reload(ds)

<module 'scvi_extensions.dataset' from '/netapp/home/mincheol/scVI-extensions/scvi_extensions/dataset/__init__.py'>

In [6]:
# Load the model
model_path = '/netapp/home/mincheol/vae_model_test.model'
vae = torch.load(model_path, map_location=lambda storage, loc: storage)

In [7]:
# Load the dataset
h5_filename = '/netapp/home/mincheol/filtered_gene_bc_matrices_h5.h5'
metadata_filename = '/ye/yelabstore2/dosageTF/tfko_140/combined/nsnp20.raw.sng.km_vb1.norm.meta.txt'
gene_dataset = ds.CropseqDataset(
    filename=h5_filename,
    metadata_filename=metadata_filename,
    use_donors=True,
    use_labels=True,
    save_path='')

File /netapp/home/mincheol/filtered_gene_bc_matrices_h5.h5 already downloaded
Preprocessing CROP-seq dataset
Number of cells kept after filtering with metadata: 173536
200900.0


MemoryError: 

In [ ]:
# Create an inference class
use_batches=True
use_cuda=True
infer = VariationalInference(
    vae, 
    gene_dataset, 
    train_size=0.9, 
    use_cuda=use_cuda,
    verbose=True,
    frequency=5)